In [1]:
from ltn_imp.parsing.parser import LTNConverter
import torch
from ltn_imp.automation.data_loaders import LoaderWrapper
from ltn_imp.automation.knowledge_base import KnowledgeBase

In [2]:
with open('test.yaml', 'r') as f:
    converter = LTNConverter(yaml_file=f)

In [3]:
converter.parse(converter.yaml["constraints"][1]).variables()

{<VariableExpression>: c,
 <VariableExpression>: flower,
 <VariableExpression>: flower2,
 <VariableExpression>: setosa,
 <VariableExpression>: versicolor,
 <VariableExpression>: virginica}

In [4]:
learnables = converter.yaml["predicates"]
predicates = list(learnables.keys())
predicates = { pred : ... for pred in predicates} # TODO: This part initializes the Neural Networks
predicates

{'Iris': Ellipsis, 'Iris_2': Ellipsis}

In [5]:
loaders = []

for pred in predicates.keys():

    values = [ list(_.values())[0] for _ in converter.yaml["predicates"][pred]["args"]]
    keys = [ list(_.keys())[0] for _ in converter.yaml["predicates"][pred]["args"]]

    variables = []
    targets = []

    for k,v in zip(keys, values):
        if k == "in":
            variables.append(v)
        else:
            targets.append(v)

    loaders.append(LoaderWrapper(loader=..., variables=variables, targets=targets)) # TODO: Decide how to map the Dataloader here

In [6]:
loaders

[<Loader>: ['flower'] -> ['type'], <Loader>: ['flower2'] -> ['type_2']]

In [7]:
rule_to_loader_mapping = {}

for rule in converter.yaml["constraints"]:
    variables = converter.parse(rule).variables()
    for v in variables:
        for loader in loaders:
            if str(v) in loader.variables or str(v) in loader.targets:
                if rule in rule_to_loader_mapping.keys():
                    rule_to_loader_mapping[rule].append(loader)
                else:
                    rule_to_loader_mapping[rule] = [loader]

In [8]:
rule_to_loader_mapping

{'forall flower. exists c. (flower[class] = c) <-> Iris(flower, c)': [<Loader>: ['flower'] -> ['type']],
 'forall flower2. Iris(flower, c) and Only_one_of_three(c, setosa, versicolor, virginica)': [<Loader>: ['flower2'] -> ['type_2'],
  <Loader>: ['flower'] -> ['type']]}